In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


train = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
test = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

Train shape: (8000, 30)
Test shape: (2845, 29)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Unnamed: 0,ID,class,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,0,1,water,637.5950,658.668,-1882.030,-1924.36,997.904,-1739.990,630.087,...,NaN,-1043.160,-1942.490,267.138,NaN,NaN,211.328,-2203.020,-1180.19,433.906
1,1,2,water,634.2400,593.705,-1625.790,-1672.32,914.198,-692.386,707.626,...,NaN,-933.934,-625.385,120.059,364.858,476.972,220.878,-2250.000,-1360.56,524.075
2,3,4,water,58.0174,-1599.160,NaN,-1052.63,NaN,-1564.630,NaN,...,-1025.88,368.622,NaN,-1227.800,304.621,NaN,369.214,-2202.120,NaN,-1343.550
3,4,5,water,72.5180,NaN,380.436,-1256.93,515.805,-1413.180,-802.942,...,-1813.95,155.624,NaN,-924.073,432.150,282.833,298.320,-2197.360,NaN,-826.727
4,7,8,water,1136.4400,NaN,NaN,1647.83,1935.800,NaN,2158.980,...,1535.00,1959.430,-279.317,-384.915,-113.406,1020.720,1660.650,-116.801,-568.05,-1357.140


In [5]:
ndvi_columns = [col for col in train.columns if col.endswith('_N')]
print(f"Number of NDVI columns: {len(ndvi_columns)}")
print(ndvi_columns[:5])

Number of NDVI columns: 27
['20150720_N', '20150602_N', '20150517_N', '20150501_N', '20150415_N']


In [6]:
train[ndvi_columns] = train[ndvi_columns].bfill(axis=1).ffill(axis=1)
test[ndvi_columns] = test[ndvi_columns].bfill(axis=1).ffill(axis=1)

print("Missing values in train NDVI columns:", train[ndvi_columns].isnull().sum().sum())
print("Missing values in test NDVI columns:", test[ndvi_columns].isnull().sum().sum())

Missing values in train NDVI columns: 0
Missing values in test NDVI columns: 0


In [7]:
def extract_features(df):
    df['ndvi_mean'] = df[ndvi_columns].mean(axis=1)
    df['ndvi_std'] = df[ndvi_columns].std(axis=1)
    df['ndvi_max'] = df[ndvi_columns].max(axis=1)
    df['ndvi_min'] = df[ndvi_columns].min(axis=1)
    df['ndvi_range'] = df['ndvi_max'] - df['ndvi_min']
    return df[['ndvi_mean', 'ndvi_std', 'ndvi_max', 'ndvi_min', 'ndvi_range']]

X_train = extract_features(train)
X_test = extract_features(test)

X_train.head()

,ndvi_mean,ndvi_std,ndvi_max,ndvi_min,ndvi_range
0,-344.393074,1041.114786,997.904,-2203.02,3200.924
1,-277.858070,944.416487,914.198,-2250.00,3164.198
2,-533.499022,1077.977950,952.178,-3162.14,4114.318
3,-308.795037,1012.486992,1232.120,-2829.40,4061.520
4,768.209963,1217.060330,2158.980,-2646.05,4805.030


In [8]:
le = LabelEncoder()
y_train = le.fit_transform(train['class'])

print("Classes:", le.classes_)
print("Encoded labels example:", y_train[:5])

Classes: ['farm' 'forest' 'grass' 'impervious' 'orchard' 'water']
Encoded labels example: [5 5 5 5 5]


In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_scaled, y_train, test_size=0.2, stratify=y_train, random_state=42)

print(f"Training set size: {X_tr.shape[0]} samples")
print(f"Validation set size: {X_val.shape[0]} samples")

Training set size: 6400 samples
Validation set size: 1600 samples


In [11]:
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=3000,
    class_weight='balanced',
    random_state=42
)

model.fit(X_tr, y_tr)

LogisticRegression(class_weight='balanced', max_iter=3000,
                   multi_class='multinomial', random_state=42)

In [12]:
y_val_pred = model.predict(X_val)

print("Classification Report on Validation Set:\n")
print(classification_report(y_val, y_val_pred, target_names=le.classes_))

Classification Report on Validation Set:

              precision    recall  f1-score   support

        farm       0.37      0.62      0.46       168
      forest       0.98      0.56      0.72      1232
       grass       0.18      0.56      0.28        39
  impervious       0.82      0.78      0.80       134
     orchard       0.02      0.83      0.03         6
       water       0.48      0.62      0.54        21

    accuracy                           0.59      1600
   macro avg       0.47      0.66      0.47      1600
weighted avg       0.87      0.59      0.68      1600



In [14]:
model.fit(X_train_scaled, y_train)
y_test_pred = model.predict(X_test_scaled)
y_test_labels = le.inverse_transform(y_test_pred)

submission = pd.DataFrame({
    'ID': test['ID'],
    'class': y_test_labels
})

submission.to_csv('submission.csv', index=False)

In [15]:
from IPython.display import FileLink
FileLink('submission.csv')


/kaggle/working/submission.csv